# 연관된 데이터 선정

In [7]:
# 프롬프트
from langchain_core.prompts import PromptTemplate

relevance_template = '''
당신은 데이터 과학자입니다. 아래는 연구 데이터 목록입니다.

각 데이터 항목은 다음 컬럼을 가지고 있습니다:
- ID: 각 데이터의 고유키
- 제목
- 설명

[목표]
주어진 연구 주제와 가장 관련성 높은 3~5개의 데이터 항목을 선택하세요. 

[Input]
연구 주제: {subject}
연구 설명: {description}

[Data]
데이터 목록:
{data}

[Output]
가장 관련성 높은 3~5개 항목의 svc_id를 JSON으로 출력해주세요.
'''

relevance_prompt = PromptTemplate.from_template(relevance_template)


In [8]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel

class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[str],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="가장 관련성이 높은 데이터 ID 리스트, 길이 최소 3개/최대 5개", 
        )
    ]

In [9]:
from langchain_openai import ChatOpenAI
import pandas as pd
import json


df = pd.read_csv('../data/search_results.csv', encoding='UTF-8', low_memory=False)

with open("../data/input_data.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)
    
subject, description = input_data['dataset_title_etc_main'], input_data['dataset_expl_etc_main']


prompt = relevance_prompt.invoke(
    {
        'subject': subject, 
        'description': description,
        'data': df[['svc_id', 'dataset_title_etc_main', 'dataset_expl_etc_main']].to_dict(orient="records"),
    }
)

# sLLM
sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

structured_sllm = sllm.with_structured_output(IDRelevance)
res = structured_sllm.invoke(prompt)

# Output
id_list = res.relevant_id
selected_data = df[df['svc_id'].isin(id_list)]

display(selected_data)

,svc_id,ctlg_type,dataset_type,ctlg_type_pc,dataset_type_pc,dataset_pub_dt_pc,dataset_access_type_pc,file_yn_pc,dataset_cc_license_pc,dataset_main_lang_pc,...,ministry_etc,pjt_nm_kor,pjt_nm_etc,pjt_mngr_kor,pjt_mngr_etc,dataset_cntrbtr_kor,dataset_cntrbtr_etc,dataset_pblshr,dataset_pric,query
1,90ca58cfe0896194588fe73299e585f1,2,2,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],Architectural Urbanism
7,b37f0c9413eeb7c45f6fe31cbe3a41ef,2,2,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],Melbourne Seoul
10,b37f0c9413eeb7c45f6fe31cbe3a41ef,2,2,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],KTA projects
16,3c81f57da77b6cd77b261d4fb1522771,2,2,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],civic architecture
17,5a27d0958baf27c7372453fe940b4db0,2,2,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],civic architecture
19,175af073d420cd3aa8b18fc8f7360e4e,2,2,dataset,해외,2024,공개,랜딩페이지이동,none,English,...,[],[],[],[],[],[],[],[],[],civic architecture
